## Импорты

In [19]:
import pandas as pd
from reader3 import Table, convertor, test_convertor
from sklearn import preprocessing
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection.tests.test_split import train_test_split
from sklearn import model_selection 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import joblib
import numpy as np

## Получение данных

In [3]:

pd.set_option('max_columns', None)


tb = Table(["01_all_users.csv","02_events_log.csv","03_lk_events_log.csv","04_is_blocked.csv","test.csv"])
hr = convertor(tb.get_all_users())
col_names = hr.columns.tolist()
print("Пример данных:")
hr

Пример данных:


,id,day_or_month_contract,blocked,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета(Event),Включение IPTV-пакета(Event),"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена Тарифа(Event),Удаление из Обращались с номеров,Смена пароля(Event),Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут(Event),Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение(Event),Отключение скидки(Event),Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж(Event),Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение(Event),Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,Информер ВК. Показ,Включение интернета на 20 минут,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение,Принято заявление и доп. соглашение по переезду,Смена пароля,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета,Выключение IPTV-пакета,Смена тарифа,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,Закрытие информатора об изменении уведомлений через ВК платформу,Включение управления услугами по паролю,Общий счет. Отсоединение реципиента,Общий счет. Отсоединение от донора,"Отключение услуги ""Стоп-тариф""",Подписка на ivi. Включение триала,Подписка на ivi. Отключение автопродления подписки,Отключение скидки,Добавление в Обращались с номеров(t),Обращение в службу заботы о клиентах(t),Выключение IPTV-пакета(Event)(t),Включение IPTV-пакета(Event)(t),"Включение услуги ""Ограничение функций ТВ""(t)",Отключение услуги Аренда ТВ-приставки(t),Оборудование(t

## Проверка полученых данных

In [4]:
hr.shape

(6506, 247)

In [5]:
hr.describe()

,id,day_or_month_contract,blocked,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета(Event),Включение IPTV-пакета(Event),"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена Тарифа(Event),Удаление из Обращались с номеров,Смена пароля(Event),Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут(Event),Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение(Event),Отключение скидки(Event),Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж(Event),Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение(Event),Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,Информер ВК. Показ,Включение интернета на 20 минут,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение,Принято заявление и доп. соглашение по переезду,Смена пароля,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета,Выключение IPTV-пакета,Смена тарифа,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,Закрытие информатора об изменении уведомлений через ВК платформу,Включение управления услугами по паролю,Общий счет. Отсоединение реципиента,Общий счет. Отсоединение от донора,"Отключение услуги ""Стоп-тариф""",Подписка на ivi. Включение триала,Подписка на ivi. Отключение автопродления подписки,Отключение скидки,Добавление в Обращались с номеров(t),Обращение в службу заботы о клиентах(t),Выключение IPTV-пакета(Event)(t),Включение IPTV-пакета(Event)(t),"Включение услуги ""Ограничение функций ТВ""(t)",Отключение услуги Аренда ТВ-приставки(t),Оборудование(t

## Изучение данных

In [6]:
hr['blocked'].value_counts()

0    5808
1     698
Name: blocked, dtype: int64

In [7]:
hr['blocked']

0       1
1       1
2       1
3       1
4       1
       ..
6501    0
6502    0
6503    0
6504    0
6505    0
Name: blocked, Length: 6506, dtype: int64

In [8]:
hr.groupby('blocked').mean()

,id,day_or_month_contract,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета(Event),Включение IPTV-пакета(Event),"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена Тарифа(Event),Удаление из Обращались с номеров,Смена пароля(Event),Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут(Event),Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение(Event),Отключение скидки(Event),Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж(Event),Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение(Event),Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,Информер ВК. Показ,Включение интернета на 20 минут,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение,Принято заявление и доп. соглашение по переезду,Смена пароля,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета,Выключение IPTV-пакета,Смена тарифа,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,Закрытие информатора об изменении уведомлений через ВК платформу,Включение управления услугами по паролю,Общий счет. Отсоединение реципиента,Общий счет. Отсоединение от донора,"Отключение услуги ""Стоп-тариф""",Подписка на ivi. Включение триала,Подписка на ivi. Отключение автопродления подписки,Отключение скидки,Добавление в Обращались с номеров(t),Обращение в службу заботы о клиентах(t),Выключение IPTV-пакета(Event)(t),Включение IPTV-пакета(Event)(t),"Включение услуги ""Ограничение функций ТВ""(t)",Отключение услуги Аренда ТВ-приставки(t),Оборудование(t),Измене

## Нормализация данных

In [4]:

hr_vars=hr.columns.values.tolist()

yy1 =['blocked','id']
Xx=[i for i in hr_vars if i not in yy1]

scaler = preprocessing.StandardScaler().fit(hr[Xx])
X_scaled = scaler.transform(hr[Xx])
print(X_scaled)
hr[Xx] = X_scaled

[[-0.85082335  5.60095576  3.80856201 ... -0.01690282 -0.014126
  -0.01239871]
 [-0.85082335  5.60095576  2.14685766 ... -0.01690282 -0.014126
  -0.01239871]
 [-0.85082335 -0.16940341 -0.34569887 ... -0.01690282 -0.014126
  -0.01239871]
 ...
 [ 1.17533211 -0.16940341  2.97770984 ... -0.01690282 -0.014126
  -0.01239871]
 [-0.85082335 -0.16940341 -0.34569887 ... -0.01690282 -0.014126
  -0.01239871]
 [ 1.17533211 -0.16940341 -0.34569887 ... -0.01690282 -0.014126
  -0.01239871]]


## Проверка нормализации данных

In [10]:
hr.describe()

,id,day_or_month_contract,blocked,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета(Event),Включение IPTV-пакета(Event),"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена Тарифа(Event),Удаление из Обращались с номеров,Смена пароля(Event),Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут(Event),Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение(Event),Отключение скидки(Event),Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж(Event),Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение(Event),Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,Информер ВК. Показ,Включение интернета на 20 минут,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение,Принято заявление и доп. соглашение по переезду,Смена пароля,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета,Выключение IPTV-пакета,Смена тарифа,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,Закрытие информатора об изменении уведомлений через ВК платформу,Включение управления услугами по паролю,Общий счет. Отсоединение реципиента,Общий счет. Отсоединение от донора,"Отключение услуги ""Стоп-тариф""",Подписка на ivi. Включение триала,Подписка на ivi. Отключение автопродления подписки,Отключение скидки,Добавление в Обращались с номеров(t),Обращение в службу заботы о клиентах(t),Выключение IPTV-пакета(Event)(t),Включение IPTV-пакета(Event)(t),"Включение услуги ""Ограничение функций ТВ""(t)",Отключение услуги Аренда ТВ-приставки(t),Оборудование(t

## Создание конечной перменой и предикторы для прогнозирования

In [5]:


y=['blocked']
y1 =['blocked','id']
X=[i for i in hr_vars if i not in y1]


## Отбор самых важных параметров методом Logistic regression model и Random Forest

In [6]:

rfe = RFECV(LogisticRegression(), step=1,min_features_to_select=5)
rfe = rfe.fit(hr[X], hr[y].values.ravel())
sp = []
print('Logistic regression model:')
for i in zip(X, rfe.support_, rfe.ranking_):
    if i[1]:
        print(i[0])
        sp.append(i[0])
        
rfe = RFECV(RandomForestClassifier(), step=1,min_features_to_select=5)
rfe = rfe.fit(hr[X], hr[y].values.ravel())
sp1 = []
print('\nRandom Forest')
for i in zip(X, rfe.support_, rfe.ranking_):
    if i[1]:
        print(i[0])
        sp1.append(i[0])


C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

...

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\mila8\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

Logistic regression model:
Отключение услуги Аренда ТВ-приставки
Операция с mac-адресом
Отключение услуги "IPTV-Бонус"
Смена IP-адреса
Смена следующего тарифа
Без границ бесплатно. Включение(Event)
Добавление IP-адреса
Удаление ip-адреса
Тайный покупатель
Принято заявление и доп. соглашение по переезду
Без границ платно. Отключение
Турбокнопка бесплатно
Бонус-скорость
Восстановление пароля
Отключение услуги "IPTV-Бонус"(t)
Смена следующего тарифа(t)
Без границ бесплатно. Включение(Event)(t)
Добавление IP-адреса(t)
Удаление ip-адреса(t)
Включение интернета на 20 минут(t)
Без границ платно. Отключение(t)
Родительский контроль. Включение(t)

Random Forest
day_or_month_contract
Добавление в Обращались с номеров
Обращение в службу заботы о клиентах
Выключение IPTV-пакета(Event)
Включение IPTV-пакета(Event)
Включение услуги "Ограничение функций ТВ"
Отключение услуги Аренда ТВ-приставки
Оборудование
Изменение истории
Продление действия тарифа
Предложение IPTV
Активация точки подключения на су

In [7]:

X_train1, X_test1, y_train1, y_test1 = train_test_split(hr[sp1],hr['blocked'] , test_size=0.15, random_state=0)

### Функция для  Cross Validation

In [11]:
def CrossValidation(modelCV,X_train2, y_train2):
     
    kfold = model_selection.KFold(n_splits=10, random_state=7,shuffle=True) 
    #modelCV = LogisticRegression()
    scoring = 'accuracy' 
    results = model_selection.cross_val_score(modelCV, X_train2, y_train2, cv=kfold, scoring=scoring)
    logregmcv =results.mean() 
    #print("10-fold cross validation average accuracy: %.3f" % (logregmcv)) 
    return logregmcv 

## Функция Random Forest 

In [26]:
def RandomForest(rf,X_train4, y_train4,y_test4,X_test4,gh):
    ranforcv = CrossValidation(rf,X_train4,y_train4)
    if gh == 'lgm':
        rf1 = rf
        rf1.fit(X_train4, y_train4)
        print(rf1)
        ranfor = accuracy_score(y_test4, rf1.predict(X_test4))
        rf_roc_auc = roc_auc_score(y_test4, rf1.predict(X_test4))
        rf_fpr, rf_tpr, rf_thresholds = roc_curve(y_test4, rf1.predict_proba(X_test4)[:,1])
        return ranfor,ranforcv,rf_roc_auc,rf_fpr,rf_tpr,rf1,rf_thresholds
    else:
        rf2 = rf
        rf2.fit(X_train4, y_train4)
        print(rf2)
        ranfor = accuracy_score(y_test4, rf2.predict(X_test4))
    
        rf_roc_auc1 = roc_auc_score(y_test4, rf2.predict(X_test4))
        rf_fpr1, rf_tpr1, rf_thresholds1 = roc_curve(y_test4, rf.predict_proba(X_test4)[:,1])
        return ranfor,ranforcv,rf_roc_auc1,rf_fpr1,rf_tpr1,rf2,rf_thresholds1
    

## Random Forest c методом Random Forest

In [27]:
finaly,finaly1,rf_roc_auc1,rf_fpr1,rf_tpr1,rf1,rf_thresholds1 = RandomForest(RandomForestClassifier(),X_train1, y_train1,y_test1,X_test1, 'rf')
print(finaly,finaly1)

RandomForestClassifier()
0.8872950819672131 0.8911392405063292


# Прогнозирование

In [14]:
hr_test = test_convertor(tb.get_all_test_user())
hr_vars1=hr_test.columns.values.tolist()

yy1 =['id']
Xx=[i for i in hr_vars1 if i not in yy1]

scaler = preprocessing.StandardScaler().fit(hr_test[Xx])
X_scaled = scaler.transform(hr_test[Xx])
hr_test[Xx] = X_scaled


268331
268507
268530
268631
268850
268906
268966
269153
269169
269249
269336
269363
269380
269526
269547
269562
269572
269997
270054
270246
270512
270669
270697
270711
270763
270848
270907
271186
271224
271301
271327
271352
271435
271545
271559
271572
271646
271677
271755
271905
271910
271974
272036
272167
272179
272197
272355
272424
272472
272518
272545
272614
272626
272642
272646
272720
272806
272824
272847
272876
273185
273193
273202
273232
273271
273309
273311
273358
273371
273380
273470
273473
273521
273552
273633
273689
273710
273751
273810
273834
273847
273849
273861
273961
273982
273984
274001
274018
274024
274055
274116
274154
274203
274209
274230
274233
274307
274336
274388
274415
274426
274438
274468
274482
274486
274540
274773
2089
6442
3803
3933
1034
21360
2434
3813
3695
22367
1261
7648
2344
5976
7574
6943
21167
5079
23135
22619
23258
4145
23381
22496
2587
23518
23247
3379
2913
3631
3522
2743
23627
23685
2409
5357
7287
3920
7257
21037
4512
2816
2710
8100
6755
3389
6024
238

In [15]:
hr_test

,id,day_or_month_contract,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета(Event),Включение IPTV-пакета(Event),"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена Тарифа(Event),Удаление из Обращались с номеров,Смена пароля(Event),Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут(Event),Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение(Event),Отключение скидки(Event),Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж(Event),Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение(Event),Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,Информер ВК. Показ,Включение интернета на 20 минут,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение,Принято заявление и доп. соглашение по переезду,Смена пароля,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета,Выключение IPTV-пакета,Смена тарифа,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,Закрытие информатора об изменении уведомлений через ВК платформу,Включение управления услугами по паролю,Общий счет. Отсоединение реципиента,Общий счет. Отсоединение от донора,"Отключение услуги ""Стоп-тариф""",Подписка на ivi. Включение триала,Подписка на ivi. Отключение автопродления подписки,Отключение скидки,Добавление в Обращались с номеров(t),Обращение в службу заботы о клиентах(t),Выключение IPTV-пакета(Event)(t),Включение IPTV-пакета(Event)(t),"Включение услуги ""Ограничение функций ТВ""(t)",Отключение услуги Аренда ТВ-приставки(t),Оборудование(t),Измене

In [16]:
hr_test['blocked'] = rf1.predict(hr_test[sp1])
fd = rf1.predict(hr_test[sp1])
 

In [17]:
filename = 'finalized_model.sav'
joblib.dump(rf1, filename)

['finalized_model.sav']

In [20]:
np.save('example_1', sp1)


## Запись в файл exit.csv

In [21]:
rf1

RandomForestClassifier()

In [70]:
hr_test.to_csv('exit.csv',columns=['id', 'blocked'], index=False)